In [16]:
import json
import torch

from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as tfidf_cosine_similarity
from torch.nn.functional import cosine_similarity

In [10]:
# Récupère les documents du json
documents = []
with open("./data/documents.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

# Extraction de mots-clés
keywords_modele = KeyBERT()
for doc in documents:
    texte = f"{doc.get('title_s', '')} {doc.get('abstract_s', '')}"
    keywords = keywords_modele.extract_keywords(texte, keyphrase_ngram_range=(1, 3), top_n=5) 
    doc['extracted_keywords'] = [kw[0] for kw in keywords]  # Ajouter les mots-clés

In [11]:
# Chargement du modèle
modele = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Extraction des informations des documents
informations = [
    f"{doc.get('title_s', '')} "  # Titre
    f"{', '.join(doc.get('keyword_s', []))} "  # Mots-clés
    f"{', '.join(doc.get('extracted_keywords', []))} "  # Mots-clés extraits
    f"Auteurs: {', '.join(doc.get('authFullName_s', []))} "  # Auteurs
    for doc in documents
]

In [12]:
# Générer les embeddings
embeddings = modele.encode(informations, convert_to_tensor=True)

# Associer chaque embedding à son document
documents_valides = [doc for doc, titre in zip(documents, informations) if titre.strip()]
documents_embeddings = [
    {"document": doc, "embedding": embedding}
    for doc, embedding in zip(documents_valides, embeddings)
]

# Calculer la similarité cosinus entre les embeddings
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(informations)

In [22]:
# Fonction qui cherche les documents les plus pertinents pour une requête
def recherche(query, tfidf_vectorizer, tfidf_matrix, embeddings, documents, model):
    # Si la requête contient des mots-clés, on effectue une recherche par mots-clés
    if detecter_recherche_par_mots_cles(query):
        return [res[0] for res in recherche_par_mots_cles(query, documents)]
    else:
        return recherche_hybride(query, tfidf_vectorizer, tfidf_matrix, embeddings, documents, model)

# Fonction qui permet de détecter si la requête est une recherche par mots-clés
def detecter_recherche_par_mots_cles(query):
    termes_indicateurs = ["liés à", "documents sur", "articles sur", "mots-clés", "thème", "traitent de", "traitant de", "concernant", "par rapport à"]
    return any(terme in query.lower() for terme in termes_indicateurs)

# Fonction qui recherche les documents par mots-clés
def recherche_par_mots_cles(query, documents):
    resultats = []
    for doc in documents:
        mots_cles = doc.get('keyword_s', []) + doc.get('extracted_keywords', []) # On se concentre sur tous les mots-clés
        similarite = sum(1 for mot in mots_cles if mot.lower() in query.lower())
        if similarite > 0:
            resultats.append((doc, similarite))
    return sorted(resultats, key=lambda x: x[1], reverse=True)[:10]  # Trier par pertinence

# Fonction qui recherche les documents avec une combinaison de similarité sémantique et TF-IDF
def recherche_hybride(query, tfidf_vectorizer, tfidf_matrix, embeddings, documents, model):
    # Recherche sémantique avec embeddings
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarites_sémantiques = cosine_similarity(query_embedding, embeddings)
    
    # Recherche TF-IDF
    tfidf_query = tfidf_vectorizer.transform([query])
    similarites_tfidf = tfidf_cosine_similarity(tfidf_query, tfidf_matrix)

    # Recherche par mots-clés
    scores_keywords = []
    for doc in documents:
        mots_cles = doc.get('keyword_s', []) + doc.get('extracted_keywords', [])
        scores_keywords.append(sum(1 for mot in mots_cles if mot.lower() in query.lower()))
    
    # Combinaison des scores
    # Attention : `similarites_tfidf` est 2D, donc on prend le vecteur [0]
    scores_combines = (0.3 * similarites_sémantiques + 0.6 * similarites_tfidf[0] + 0.1 * torch.tensor(scores_keywords))

    # Tri des résultats par pertinence
    indices_tries = scores_combines.argsort(descending=True)  # Indices triés par score
    return [documents[i] for i in indices_tries[:10]]


In [23]:
# Exemple de requête
query = "Apprentissage par renforcement"
resultats = recherche_hybride(query, tfidf_vectorizer, tfidf_matrix, embeddings, documents, modele)

# Afficher les résultats
for res in resultats:
    print(f"Titre: {res['title_s']}, Mots-clés: {res['keyword_s']}, Auteurs: {res['authFullName_s']}")

Titre: ['Vers la généralisation de l’apprentissage par renforcement', 'Toward the generalization of reinforcement learning'], Mots-clés: ['Reinforcement learning', 'Multimodal agents', 'Exploration', 'General-purpose agent', 'Sparsely rewarded environments', 'Apprentissage par renforcement', 'Agents multimodaux', 'Exploration', 'Agent à usage général', 'Environnements à récompenses rares'], Auteurs: ['Quentin Gallouédec']
Titre: ['Étude de la motivation intrinsèque en apprentissage par renforcement'], Mots-clés: ['Reinforcement learning', 'Intrinsic motivation', 'Curiosity', 'Knowledge acquisition', 'Options', 'Generation of objectives', 'Meta-reward', 'Apprentissage par renforcement', 'Motivation intrinsèque', 'Curiosité', 'Acquisition de connaissances', 'Empowerment', 'Options', "Génération d'objectifs"], Auteurs: ['Arthur Aubret', 'Laëtitia Matignon', 'Salima Hassas']
Titre: ['Learning increasingly complex skills through deep reinforcement learning using intrinsic motivation', "Appr

C:\Users\tomhu\AppData\Local\Temp\ipykernel_8488\2562472673.py:42: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  scores_combines = (0.3 * similarites_sémantiques + 0.6 * similarites_tfidf[0] + 0.1 * torch.tensor(scores_keywords))
